In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import community as community_louvain
import numpy as np
import networkx as nx
from community import community_louvain
from random import sample
from scipy.integrate import odeint
import scipy.sparse as sp
import pandas as pd
import numpy.linalg as lg
import operator
%matplotlib inline
from tabulate import tabulate
from networkx.algorithms import community

In [ ]:
G=nx.read_weighted_edgelist('ColoSpg.txt')# Colorado Springs dataset
D=nx.read_weighted_edgelist('Drugs.txt')
Z=nx.read_gml("dolphins.gml")
F=nx.read_gml("lesmis.gml")
ER=nx.erdos_renyi_graph(500,0.024,seed=1)
air=nx.read_weighted_edgelist('USAir 97.txt')
rd=nx.read_weighted_edgelist('road2.txt')
BA=nx.barabasi_albert_graph(100,3,seed=1)
air2=nx.read_edgelist('airline.txt')

In [ ]:
def matrix(g):# graph matrices
    return nx.to_numpy_matrix(g

# Non-Linear Dynamical System

In [ ]:
def epi_step(A, p, n, gamma, beta):
    q = np.zeros(n)
    for i in range(n):
        z = 1
        f,f1,v=sp.find(A[:,i])# returns the non-zero elements in a sparse matrix
        for j in f:
            z = z*(1-beta*A[i,j]*p[j]) #zeta
        q[i] = (1-p[i])*z+gamma*p[i]*z # probability of not being infected
    p = 1-q 
    
    return p# probability of  being infected

In [21]:
def trial(A, gamma, beta, m, init):
    t = np.zeros(m)
    n = len(A)
    p=init
    for i in range(m):
        t[i] = sum(p)
        p = epi_step(A, p, n, gamma, beta)
    return t,p

In [346]:
def epidemic(g): 
    """ Function that returns fraction of infections at every time-step
    """
    #for i in range(5):
    m=100
    gamma=.08
    beta=.14
    init = np.zeros(len(matrix(g)))
    init[0] = 1
    t,p1 = trial(matrix(g),gamma,beta,m,init)
    a=plt.plot(range(m),t/len(matrix(g)),'x',label=('no vaccination'))
    #plt.grid()
    #b=plt.legend()
    return a

# SIS Model -differential equation

In [1]:
def my_SIS(g,beta,gamma,t,ax1):   
    N = len(g.nodes()) # Total population
    I0 = 1 #initial infected 
    S0 = N - I0 #initial susceptibles


    def SIS(y, t, N, beta, gamma):
        S, I = y
        dSdt = -beta * S * I / N +gamma*I


        dIdt = beta * S * I / N - gamma * I
        return dSdt, dIdt


    y0 = S0, I0 #Initial condituions

    diff = odeint(SIS, y0, t, args=(N, beta, gamma))
    S, I = diff.T


    a=ax1.plot(t, I/N, 'r',label=("SIS",round(max(I/N),2)))
    
#     plt.xlabel('Timesteps')
#     plt.ylabel('fraction of infectives')
#     plt.ylim(0,1)

#     plt.show()
    return a


# Kephart White Model


In [ ]:
#average degree of networks
def ave_deg(g):
    degrees=dict(g.degree())
    list_deg=list(degrees.values())
    av_k=np.mean(list_deg)
    return av_k

In [ ]:
def my_KW(g,beta,gamma,t,ax1): # The Kephart and White Model
    N=len(g.nodes())
        
    def kephart(eta,t): 

        return beta*ave_deg(g)*eta*(1-eta/N)-gamma*eta

    eta0=1
    y=odeint(kephart,eta0,t)
    a=ax1.plot(t, y/N, 'k',label=("KW",(max(y/N),2)))
    return a

In [ ]:
# set width of bar
barWidth = 0.25
fig = plt.subplots(figsize =(12, 8))
 
# set height of bar
NLDS = [0.57,0.79, .81, 0.8,0.92,.85]
KW = [0.53, 0.85, 0.81, 0.85,.87,.83]
SIS = [0, 0, 0.02, 0.01,0,0]
 
# Set position of bar on X axis
br1 = np.arange(6)
br2 = [x + barWidth for x in br1]
br3 = [x + barWidth for x in br2]
 
# Make the plot
plt.bar(br1, NLDS, color ='r', width = barWidth,
        edgecolor ='grey', label ='NLDS')
plt.bar(br2, KW, color ='g', width = barWidth,
        edgecolor ='grey', label ='KW')
plt.bar(br3, SIS, color ='b', width = barWidth,
        edgecolor ='grey', label ='SIS')
 
# Adding Xticks
#plt.title("Beta=0.14 Gamma=0.14")
plt.legend()
plt.grid()
plt.xlabel('Data Sets', fontweight ='bold', fontsize = 15)
plt.ylabel('Fraction Infected', fontweight ='bold', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(NLDS))],
        ['Colorado Springs', 'Drugs', 'Dolphin', 'Les Miserables',"ER",'BA'])
plt.show()

https://www.geeksforgeeks.org/bar-plot-in-matplotlib/

I edited this code by adding my own values and titles.

# VACCINATION

### NB: For matrices, we return A

### NB: For graphs we return r

# Vaccinating hubs

In [ ]:

def remove_hubs(g):
    deg=np.array(list(dict(g.degree()).values())).astype(float) 
    deg_sort=sorted(deg, reverse=True)#sorting degrees
    mx=deg_sort[:int(0.1*len(g.nodes()))]# 10% of nodes
    a=[]
    for nodes in g.nodes(): # returning hubs
        if g.degree(nodes) in mx:
            a.append(nodes) 
    
    
    gnow=g.copy()
    gnow.remove_nodes_from(a) #removing hubs

    A=nx.to_numpy_matrix(gnow)
    gnow=nx.from_numpy_matrix(A)
    
    #infecting largest component to get maximum infection
    component_subgraph_list = list(nx.connected_component_subgraphs(gnow))
    largest_component= max(component_subgraph_list,key=len)
                                   
    t=nx.degree_centrality(largest_component) #infecting  nodes with highest deg
    sorted_tuples = sorted(t.items(), key=operator.itemgetter(1),reverse=True)
    sorted_dict = {k: v for k, v in sorted_tuples}

    p=list(sorted_dict.keys()) 
    q=p[:5] # averaging on 5 graphs starting at different  nodes
    s=[]
    for i in q: #ndls
        m=100
        gamma=.08
        beta=.14
        init = np.zeros(len(A))
        init[int(i)] = 1
        t,p1 = trial(A,gamma,beta,m,init)
        s.append(t)
        c=np.mean(s, axis=0)
    r=plt.plot(range(m),c/len( matrix(g)),'r',label=('hubs_vaccine'))
# #     plt.figure(figsize=(18,18))

#     b=nx.draw_networkx(gnow)
#     plt.axis('off')
#     #plt.savefig("pth2.png")
#     plt.show()
    return r


# Random Vaccination

In [ ]:
def remove_random(g):
    ran= sample(list(g.nodes()), int(0.1*len(g.nodes()))) #seleting random nodes
    
    gnow=g.copy()
    gnow.remove_nodes_from(ran) #removing nodes

    A=nx.to_numpy_matrix(gnow)
    gnow=nx.from_numpy_matrix(A) 
    
    #infecting the largest component
    component_subgraph_list = list(nx.connected_component_subgraphs(gnow))
    largest_component= max(component_subgraph_list,key=len)
                                   
    t=nx.degree_centrality(largest_component)
    sorted_tuples = sorted(t.items(), key=operator.itemgetter(1),reverse=True)
    sorted_dict = {k: v for k, v in sorted_tuples}

    p=list(sorted_dict.keys()) 
    q=p[:5] # getting 5 starting points
    s=[]
    for i in q:
        m=100
        gamma=.08
        beta=.14
        init = np.zeros(len(A))
        init[int(i)] = 1
        t,p1 = trial(A,gamma,beta,m,init)
        s.append(t)
        c=np.mean(s, axis=0)#averaging over the 5 graphs
    r=plt.plot(range(m),c/len( matrix(g)),'b',label=('random_vaccine'))

    return r
#

 
# Node Betweenness

In [ ]:
def remove_node_btwn(g):
    d=nx.betweenness_centrality(g) #
    sorted_tuples = sorted(d.items(), key=operator.itemgetter(1),reverse=True)
    sorted_dict = {k: v for k, v in sorted_tuples}

    e=list(sorted_dict.keys()) 
    f=e[:int(0.1*len(g.nodes()))] # 10% nodes with highest betweenness
    
    gnow=g.copy()
    gnow.remove_nodes_from(f)

    A=nx.to_numpy_matrix(gnow)
    gnow=nx.from_numpy_matrix(A) 
    
    # infecting central node in largest component
    component_subgraph_list = list(nx.connected_component_subgraphs(gnow))
    largest_component= max(component_subgraph_list,key=len)
                                   
    t=nx.degree_centrality(largest_component)
    sorted_tuples = sorted(t.items(), key=operator.itemgetter(1),reverse=True)
    sorted_dict = {k: v for k, v in sorted_tuples}

    p=list(sorted_dict.keys()) 
    q=p[:5]
    s=[]
    for i in q:
        m=100
        gamma=.08
        beta=.14
        init = np.zeros(len(A))
        init[int(i)] = 1
        t,p1 = trial(A,gamma,beta,m,init)
        s.append(t)
        c=np.mean(s, axis=0)
    r=plt.plot(range(m),c/len( matrix(g)),'k',label=('node_betweenness_vaccine'))
# #     plt.figure(figsize=(18,18))

#     b=nx.draw_networkx(gnow)
#     plt.axis('off')
#     #plt.savefig("pth2.png")
#     plt.show()
    return r

# Eigen Centrality

In [ ]:
def remove_eigen(g):
    d=nx.eigenvector_centrality_numpy(g) # sorting nodes using eigenvecctor centrality
    sorted_tuples = sorted(d.items(), key=operator.itemgetter(1),reverse=True)
    sorted_dict = {k: v for k, v in sorted_tuples}

    e=list(sorted_dict.keys()) 
    f=e[:int(0.5*len(g.nodes()))] #highest 10%
    
    gnow=g.copy()
    gnow.remove_nodes_from(f)
    A=nx.to_numpy_matrix(gnow)
    gnow=nx.from_numpy_matrix(A) 
    # infecting central node in largest component
    component_subgraph_list = list(nx.connected_component_subgraphs(gnow))
    largest_component= max(component_subgraph_list,key=len)
                                   
    t=nx.degree_centrality(largest_component)
    sorted_tuples = sorted(t.items(), key=operator.itemgetter(1),reverse=True)
    sorted_dict = {k: v for k, v in sorted_tuples}

    p=list(sorted_dict.keys()) 
    q=p[:5]
    s=[]
    for i in q:
        m=100
        gamma=.08
        beta=.14
        init = np.zeros(len(A))
        init[int(i)] = 1
        t,p1 = trial(A,gamma,beta,m,init)
        s.append(t)
        c=np.mean(s, axis=0)
    r=plt.plot(range(m),c/len( matrix(g)),'g',label=('eigen_vaccine'))
# #     plt.figure(figsize=(18,18))

#     b=nx.draw_networkx(gnow)
#     plt.axis('off')
#     #plt.savefig("pth2.png")
#     plt.show()

    return r


# Acquaintance Vaccination

In [ ]:
def acquintance(g):
    a=sample(list(g.nodes()), int(0.1*len(g.nodes())))# selecting random node
    k=[]
    for i in a: #selecting neighbour of the random node
        b=sample(list(g.neighbors(i)), 1)
        k.append(b)
    f = []
    for sublist in k: #unpacking list of lists
        for node in sublist:
            f.append(node)
    gnow=g.copy()
    gnow.remove_nodes_from(f)

    A=nx.to_numpy_matrix(gnow)
    gnow=nx.from_numpy_matrix(A) 
    #infecting the largest component
    component_subgraph_list = list(nx.connected_component_subgraphs(gnow))
    largest_component= max(component_subgraph_list,key=len)
                                   
    t=nx.degree_centrality(largest_component)
    sorted_tuples = sorted(t.items(), key=operator.itemgetter(1),reverse=True)
    sorted_dict = {k: v for k, v in sorted_tuples}

    p=list(sorted_dict.keys()) 
    q=p[:5]
    s=[]
    for i in q:
        m=100
        gamma=.08
        beta=.14
        init = np.zeros(len(A))
        init[int(i)] = 1
        t,p1 = trial(A,gamma,beta,m,init)
        s.append(t)
        c=np.mean(s, axis=0)
    r=plt.plot(range(m),c/len( matrix(g)),'gray',label=('random_acquintance_vaccine'))
#     plt.figure(figsize=(18,18))

#     b=nx.draw_networkx(gnow)
#     plt.axis('off')
#     #plt.savefig("pth2.png")
#     plt.show()
    return r

# Plotting

In [ ]:
def my_graphs(g):
    plt.clf()
    fig1=plt.figure()
    ax1=fig1.add_subplot(111)
    remove_hubs_edges(g)
    rand_eg_removed(g)
    max_edge_btwn(g)
    max_hub_edge_btwn(g)
    epidemic(g)
    ax1.grid()
    plt.xlabel('timesteps')
    plt.ylabel(' fraction infected')
    plt.legend()
    plt.show()
    return ax1

# SOCIAL DISTANCING

# Most connected edges

In [ ]:

def remove_hubs_edges(g): #removing hubs from line graph
    
    c=nx.line_graph(g)
    #sorting degrees
    deg=np.array(list(dict(c.degree()).values())).astype(float)
    deg_sort=sorted(deg, reverse=True)
    mx=deg_sort[:int(0.1*len(c.nodes()))]
    a=[]
    for nodes in c.nodes():
        if c.degree(nodes) in mx:
            a.append(nodes) 
    
    
    gnow=g.copy()
    gnow.remove_edges_from(a)

    A=nx.to_numpy_matrix(gnow)
    gnow=nx.from_numpy_matrix(A)
    component_subgraph_list = list(nx.connected_component_subgraphs(gnow))
    largest_component= max(component_subgraph_list,key=len)
                                   
    t=nx.degree_centrality(largest_component)
    sorted_tuples = sorted(t.items(), key=operator.itemgetter(1),reverse=True)
    sorted_dict = {k: v for k, v in sorted_tuples}

    p=list(sorted_dict.keys()) 
    q=p[:5]
    s=[]
    for i in q:
        m=100
        gamma=.08
        beta=.14
        init = np.zeros(len(A))
        init[int(i)] = 1
        t,p1 = trial(A,gamma,beta,m,init)
        s.append(t)
        j=np.mean(s, axis=0)
    r=plt.plot(range(m),j/len(matrix(g)),'cyan',label=('max_connected_edges'))
#     plt.figure(figsize=(18,18))
   # plt.grid()
#     b=nx.draw_networkx(gnow)
#     plt.axis('off')
#     #plt.savefig("pth2.png")
#     plt.show()
    
    return r

# Random

In [ ]:
def rand_eg_removed(g):
    rand_eg=sample(list(g.edges()), int(0.1*len(g.edges())))
    gnow=g.copy()
    gnow.remove_edges_from(rand_eg) # removing random edges

    A=nx.to_numpy_matrix(gnow)
    gnow=nx.from_numpy_matrix(A) 
    #infecting the largest component
    component_subgraph_list = list(nx.connected_component_subgraphs(gnow))
    largest_component= max(component_subgraph_list,key=len)
    #infecting nodes with highest degrees and averaging                              
    t=nx.degree_centrality(largest_component)
    sorted_tuples = sorted(t.items(), key=operator.itemgetter(1),reverse=True)
    sorted_dict = {k: v for k, v in sorted_tuples}

    p=list(sorted_dict.keys()) 
    q=p[:5]
    s=[]
    for i in q:
        m=100
        gamma=.08
        beta=.14
        init = np.zeros(len(A))
        init[int(i)] = 1
        t,p1 = trial(A,gamma,beta,m,init)
        s.append(t)
        c=np.mean(s, axis=0)
    r=plt.plot(range(m),c/len( matrix(g)),'p',label=('random_edges'))
#     plt.figure(figsize=(18,18))
#     b=nx.draw_networkx(gnow)
#     plt.axis('off')
#     #plt.savefig("pth2.png")
#     plt.show()
    return r

# Edge Betweenness

In [ ]:
def max_edge_btwn(g): #removing 10% edges with highest edge betweenness
    d=nx.edge_betweenness_centrality(g)
    sorted_tuples = sorted(d.items(), key=operator.itemgetter(1),reverse=True)
    sorted_dict = {k: v for k, v in sorted_tuples}

    e=list(sorted_dict.keys()) 

    t=e[:int(0.88*len(g.edges()))] #10% edges with highest edge betweenness
    
    k=g.copy()
    k.remove_edges_from(t)
    A=nx.to_numpy_matrix(k)
    k=nx.from_numpy_matrix(A)
    
    component_subgraph_list = list(nx.connected_component_subgraphs(k)) #infecting largest component
    largest_component= max(component_subgraph_list,key=len)
                                   
    h=nx.degree_centrality(largest_component)
    sorted_tuples = sorted(h.items(), key=operator.itemgetter(1),reverse=True)
    sorted_dict = {k: v for k, v in sorted_tuples}

    p=list(sorted_dict.keys()) 
    q=p[:5]
    s=[]
    for i in q:
        m=100
        gamma=.08
        beta=.14
        init = np.zeros(len(A))
        init[int(i)] = 1
        t,p1 = trial(A,gamma,beta,m,init)
        s.append(t)
        c=np.mean(s, axis=0)
    r=plt.plot(range(m),c/len( matrix(g)),'gold',label=('edge_betweenness'))
#      plt.figure(figsize=(18,18))

#     b=nx.draw_networkx(gnow)
#     plt.axis('off')
#     #plt.savefig("pth2.png")
#     plt.show()
    return r

# Removing High betweenness edges from hubs

In [ ]:
def hubs(g): 
    """ten% nodes with the highest degree
    """
    deg=np.array(list(dict(g.degree()).values())).astype(float) 
    deg_sort=sorted(deg, reverse=True)#sorting degrees
    mx=deg_sort[:int(0.1*(len(g.nodes())))]
    a=[]
    for nodes in g.nodes:
        if g.degree(nodes) in mx:
            a.append(nodes)   
    return a

def highest_btwn_hubs(g): #edges with highest betweenness from hubs
    t=nx.edge_betweenness(g)
    s=[]
    u=[]
    for i in hubs(g): # edges of hubs
        r=list(g.edges([i]))
        s.append(r)
    def get_key(val):#betweenness of edges
        for key, value in t.items():
            if val == key or (key[1],key[0])==val:
                return key,value
    k=list(t.keys())
    my_btn = [[] for i in range(len(hubs(g)))] #hub_edges  and their edge_betweenness
    for i,j in enumerate(s):
        for k in j:
            y=get_key(k)
            my_btn[i].append(y)
    def maxi_edi(x): # returning edges with maximum betweenness
        maxi = max([i[1] for i in x])
        return [i for i in x if i[1]==maxi]
    for i in my_btn:
        u.append(maxi_edi(i))
    
    return u # LIST OF LISTS OF ALL HUB EDGES WITH THEIR MAXIMUM BETWEENNESS

In [4]:
def hub_edge_btwn(g): #unpacking list to get only the edges
    a = []
    for sublist in highest_btwn_hubs(g):
        for item in sublist:
            a.append(item[0])
    return(a)

https://www.codegrepper.com/code-examples/python/python+unpack+list+of+lists

In [ ]:
def max_hub_edge_btwn(g): #removing edges  with highest betweenness from hubs
    t=hub_edge_btwn(g)
    
    k=g.copy()
    k.remove_edges_from(t)
    A=nx.to_numpy_matrix(k)
    k=nx.from_numpy_matrix(A)
    
    component_subgraph_list = list(nx.connected_component_subgraphs(k))
    largest_component= max(component_subgraph_list,key=len)
                                   
    h=nx.degree_centrality(largest_component)
    sorted_tuples = sorted(h.items(), key=operator.itemgetter(1),reverse=True)
    sorted_dict = {k: v for k, v in sorted_tuples}

    p=list(sorted_dict.keys()) 
    q=p[:5]
    s=[]
    for i in q:
        m=100
        gamma=.08
        beta=.14
        init = np.zeros(len(A))
        init[int(i)] = 1
        t,p1 = trial(A,gamma,beta,m,init)
        s.append(t)
        c=np.mean(s, axis=0)
    r=plt.plot(range(m),c/len( matrix(g)),'m',label=('edge_betweenness_hubs'))
#     plt.figure(figsize=(18,18))

#     b=nx.draw_networkx(k)
#     plt.axis('off')
#     #plt.savefig("pth2.png")
#     plt.show()
    return r

In [ ]:
def my_graphs2(g):
    plt.clf()
    fig1=plt.figure()
    ax1=fig1.add_subplot(111)
    epidemic(g)
    remove_hubs_edges(g)
    rand_eg_removed(g)
    max_edge_btwn(g)
    max_hub_edge_btwn(g)
    ax1.grid()
    plt.xlabel('timesteps')
    plt.ylabel(' fraction infected')
    plt.legend()
    plt.show()
    return ax1

# Vaccination vs Social Distancing

In [ ]:
#PLOTTING
def my_graphs3(g):
    plt.clf()
    fig1=plt.figure()
    ax1=fig1.add_subplot(111)
    remove_hubs(g)
    remove_random(g)
    remove_node_btwn(g)
    remove_eigen(g)
    epidemic(g)
    acquintance(g)
    remove_hubs_edges(g)
    rand_eg_removed(g)
    max_edge_btwn(g)
    max_hub_edge_btwn(g)
    ax1.grid()
    plt.xlabel('timesteps')
    plt.ylabel(' fraction infected')
    plt.legend()
    plt.show()
    return ax1

# Degree Distribution

In [ ]:
def deg_seq(g):  
    pk=[i/g.number_of_nodes() for i in nx.degree_histogram(g)]
    return pk

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(12, 5))
axes[0,0].plot(deg_seq(F), 'b-',label='Les Mis')

axes[0,1].plot(deg_seq(ER), 'r-',label='ER')

axes[0,2].plot(deg_seq(G), 'g',label='Colorado Springs')

axes[0,3].plot(deg_seq(D), 'm',label='Drugs')

axes[1,0].plot(deg_seq(Z), 'k',label='Dolphin')
axes[1,1].plot(deg_seq(BA), 'y',label='Barabasi Albert')
axes[1,2].plot(deg_seq(air2), 'brown',label='Airlines')
axes[1,3].plot(deg_seq(rd), 'cyan',label='Road')

for ax in axes.flat:
    ax.set(xlabel='degrees', ylabel='fraction of nodes')
    ax.legend()
plt.show()

For subplots I used this page as a guide.

https://matplotlib.org/devdocs/gallery/subplots_axes_and_figures/subplots_demo.html


# network properties

In [ ]:
# CLUSTERING
nx.average_clustering(g)

In [ ]:
# DENSITY d=2m/n(n-1)

nx.density(g)

In [ ]:
# SHORTEST PATH LENGTH

nx.average_shortest_path_length(g) #shortest path

In [ ]:
def eigenvals(g): #eigenvalues of adjacency matrix of network
    return round(max(np.linalg.eigvals(matrix(g))).real,3)

# Thresholds

In [ ]:
def remove_hubs(g):
    deg=np.array(list(dict(g.degree()).values())).astype(float) 
    deg_sort=sorted(deg, reverse=True)#sorting degrees
    mx=deg_sort[:int(0.1*len(g.nodes()))]# 10% of nodes
    a=[]
    for nodes in g.nodes(): # returning hubs
        if g.degree(nodes) in mx:
            a.append(nodes) 
    
    
    gnow=g.copy()
    gnow.remove_nodes_from(a) #removing hubs

    A=nx.to_numpy_matrix(gnow)
    gnow=nx.from_numpy_matrix(A)
    return  round(max(np.linalg.eigvals(A)).real,3) #returns largest eigenvalue

In [ ]:
def hubs(g):
    a=(remove_hub(g), remove_hubsa(g), remove_hubsb(g),remove_hubsc(g),remove_hubsd(g),remove_hubse(g),
       remove_hubsf(g),remove_hubsg(g),remove_hubsh(g))
    return a

NB: Remove_hubs(a,b,c,d,e,f,g,h)(g) removes different fraction of nodes 

In [ ]:
"""returns s when nodes are removed
"""
def thresh(g):
    w=[]
    beta=0.14
    gamma=0.08
    r=(beta/gamma)
    d=list(hubs(g))
    for i in range(len(hubs(g))):
        s=r*d[i]
        w.append(s)
    return w  

In [ ]:
def max_edge_btwna(g): #removing 10% edges with highest edge betweenness
    d=nx.edge_betweenness_centrality(g)
    sorted_tuples = sorted(d.items(), key=operator.itemgetter(1),reverse=True)
    sorted_dict = {k: v for k, v in sorted_tuples}

    e=list(sorted_dict.keys()) 

    t=e[:int(0*len(g.edges()))] #10% edges with highest edge betweenness
    
    k=g.copy()
    k.remove_edges_from(t)
    A=nx.to_numpy_matrix(k)
    return  round(max(np.linalg.eigvals(A)).real,3)

In [ ]:
def edges(g):
    a=(max_edge_btwna(g),max_edge_btwnb(g),max_edge_btwnc(g),max_edge_btwnd(g),max_edge_btwne(g),
      max_edge_btwnf(g),max_edge_btwng(g))
    return a

In [ ]:
def thresh3(g):
    w=[]
    beta=0.14
    gamma=0.08
    r=(beta/gamma)
    d=list(edges(g))
    for i in range(len(edges(g))):
        s=r*d[i]
        w.append(s)
    return w  

# Differences in eigenvalues

In [6]:
def eigenvalue_diff(g):
    x=[max(np.linalg.eigvals(matrix(g)))] # largest eigenvalue of adjacency matrix
    
    # Eigenvalues after vaccination strategies
    y=[max(np.linalg.eigvals(remove_hubs(g))),max(np.linalg.eigvals(remove_random(g))),max(np.linalg.eigvals(remove_node_btwn(g))),max(np.linalg.eigvals(remove_eigen(g))),max(np.linalg.eigvals(acquintance(g)))]
    h=[]
    for i in range(len(y)): #getting difference of largest eigenvalue after vaccination
        k=(x[0]-y[i])
        h.append(round(k.real,3))
    return h

# Communities

In [ ]:
def my_partition(g): #partition networks
    partition = community_louvain.best_partition(g)

# draw the graph
    pos = nx.spring_layout(g)
    # color the nodes according to their partition
    cmap = cm.get_cmap('viridis', max(partition.values()) + 1)

    plt.figure(figsize=(18,18))
    nx.draw_networkx_nodes(g, pos, partition.keys(), node_size=60,
                           cmap=cmap, node_color=list(partition.values()))
    a=nx.draw_networkx_edges(g, pos, alpha=0.5)
    plt.show()
    return a, community_louvain.modularity(partition,g) #returns graph and modularity score

In [ ]:
def my_communities(g):
    partition = community_louvain.best_partition(g) #detecting communities 

    # draw the graph
    pos = nx.spring_layout(g)
    # color the nodes according to their partition
    cmap = cm.get_cmap('viridis', max(partition.values()) + 1)
    #print(partition)    
    dic={}
    for nodes in partition.keys():
        if partition[nodes] in dic.keys(): #value of key
            dic[partition[nodes]].append(nodes)# key appending value
        else:
            dic[partition[nodes]]=[nodes]
    sub={}
    for community in dic.keys():
        sub[community]=g.subgraph(dic[community]) 
        
    len_sub={community:len(sub[community]) for community in sub.keys()}   
    #print(len_sub)
#     plt.figure(figsize=(40,40))   
#     for index,community in enumerate(dic.keys()):
#         plt.subplot(len(sub),1,index+1)
#         r= nx.draw_networkx(sub[community])

    for community in sub.keys():
        if len_sub[community]==max(len_sub.values()): # getting the largest community
            p=sub[community]
            #print(len(sub[community]))
            #d=nx.draw_networkx(p)
            break
    e=nx.to_numpy_matrix(p)
    return p #returning largest community

In [ ]:
def my_communities2(g): #
    partition = community_louvain.best_partition(g)

    # draw the graph
    pos = nx.spring_layout(g)
    # color the nodes according to their partition
    cmap = cm.get_cmap('viridis', max(partition.values()) + 1)
    #print(partition)    
    dic={}
    for nodes in partition.keys():
        if partition[nodes] in dic.keys(): #value of key
            dic[partition[nodes]].append(nodes)# key appending value
        else:
            dic[partition[nodes]]=[nodes]
    sub={}
    for community in dic.keys():
        sub[community]=g.subgraph(dic[community]) 
        
    len_sub={community:len(sub[community]) for community in sub.keys()}   
    #print(len_sub)
#     plt.figure(figsize=(40,40))   
#     for index,community in enumerate(dic.keys()):
#         plt.subplot(len(sub),1,index+1)
#         r= nx.draw_networkx(sub[community])

    for community in sub.keys():
        if len_sub[community]==max(len_sub.values()): # getting the largest community
            p=sub[community]
            #print(len(sub[community]))
            #d=nx.draw_networkx(p)
            break
    t=nx.degree_centrality(p) #infecting  nodes with highest deg
    sorted_tuples = sorted(t.items(), key=operator.itemgetter(1),reverse=True)
    sorted_dict = {k: v for k, v in sorted_tuples}

    l=list(sorted_dict.keys()) 
    q=l[0] # averaging on 5 graphs starting at different  nodes
    s=[]
    #for i in q: #ndls
    m=100
    gamma=.08
    beta=.14
    init = np.zeros(len(matrix(g)))
    init[0] = 1
    t,p1 = trial(matrix(g),gamma,beta,m,init)
    s.append(t)
    c=np.mean(s, axis=0)
    r=plt.plot(range(m),t/len(matrix(g)),'cyan',label=('population', round(max(t/len(matrix(g))),2)))
    plt.legend() 
    plt.xlabel('timesteps')
    plt.ylabel('fraction of infectives')
    return r
    

In [ ]:
def epidemicom(g): # infecting the largest community
    
    m=100
    gamma=.08
    beta=.14
    init = np.zeros(len(matrix(my_communities(g))))
    init[0] = 1
    t,p1 = trial(matrix(my_communities(g)),gamma,beta,m,init)
    print(t/len(matrix(my_communities(g))))
    print(len(matrix(my_communities(g))))
    a=plt.plot(range(m),t/len(matrix(my_communities(g))),'x',label=('community',round(max(t/len(matrix(my_communities(g)))),2)))
    b=plt.legend()
    plt.xlabel('timesteps')
    plt.ylabel('fraction of infectives')
    return a,b